# Example for Patch Prediction Models

<a href="https://colab.research.google.com/github/TIA-Lab/tiatoolbox/blob/master/examples/example_patchprediction.ipynb" target="_blank"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<a href="https://kaggle.com/kernels/welcome?src=https://github.com/TIA-Lab/tiatoolbox/blob/master/examples/example_patchprediction.ipynb" target="_blank"><img src="https://kaggle.com/static/images/open-in-kaggle.svg" alt="Open In Colab"/></a>

## Prerequisites

*Note*: This notebook assumes that `tiatoolbox` has already been installed. If it isn't, you can install it to your python environment by following guideline from https://github.com/TIA-Lab/tiatoolbox or using the cell below.

In [ ]:
!apt-get -y install libopenjp2-7-dev libopenjp2-tools openslide-tools

In [ ]:
!apt-get -y install libopenjp2-7-dev libopenjp2-tools openslide-tools

In [1]:
!pip install tiatoolbox

     |████████████████████████████████| 52 kB 1.7 MB/s eta 0:00:011
  Using cached click-7.1.2-py2.py3-none-any.whl (82 kB)
     |████████████████████████████████| 2.8 MB 14.8 MB/s eta 0:00:01
     |████████████████████████████████| 6.3 MB 48.6 MB/s eta 0:00:01
     |████████████████████████████████| 11.5 MB 141.3 MB/s eta 0:00:01
  Using cached cycler-0.10.0-py2.py3-none-any.whl (6.5 kB)
  Using cached kiwisolver-1.3.1-cp36-cp36m-manylinux1_x86_64.whl (1.1 MB)
     |████████████████████████████████| 50.4 MB 13.6 MB/s eta 0:00:01
  Using cached openslide_python-1.1.2-cp36-cp36m-linux_x86_64.whl
  Using cached PyYAML-5.4.1-cp36-cp36m-manylinux1_x86_64.whl (640 kB)
  Using cached scikit_image-0.17.2-cp36-cp36m-manylinux1_x86_64.whl (12.4 MB)
  Using cached imageio-2.9.0-py3-none-any.whl (3.3 MB)
     |████████████████████████████████| 1.6 MB 147.8 MB/s eta 0:00:01
  Using cached decorator-4.4.2-py2.py3-none-any.whl (9.2 kB)
  Using cached PyWavelets-1.1.1-cp36-cp36m-manylinux1_x86_64.whl